In [ ]:
# !pip install qiskit


In [ ]:
# ✅ Quantum Teleportation Simulation (Final Clean Version)
# Works perfectly in Google Colab with Qiskit 1.x

!pip install qiskit qiskit-aer pylatexenc --quiet

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.quantum_info import Statevector, partial_trace, DensityMatrix
import matplotlib.pyplot as plt
import os, importlib

# Ensure pylatexenc loads cleanly
importlib.invalidate_caches()
os.environ["PYLATEXENC_LATEX_RUN"] = "0"

# ─────────────────────────────────────────────
# Step 1 – Build Quantum Teleportation Circuit
# ─────────────────────────────────────────────
qc = QuantumCircuit(3, 2)

# Prepare arbitrary state to teleport (Alice’s qubit)
qc.h(0)
qc.t(0)
qc.barrier()

# Create entanglement (Bell pair between Alice & Bob)
qc.h(1)
qc.cx(1, 2)
qc.barrier()

# Alice’s Bell measurement
qc.cx(0, 1)
qc.h(0)
qc.barrier()

# Measure Alice’s qubits → classical bits
qc.measure([0, 1], [0, 1])
qc.barrier()

# Bob’s correction based on classical bits
qc.cx(1, 2)
qc.cz(0, 2)

# Draw circuit (safe fallback)
try:
    qc.draw("mpl")
    plt.show()
except Exception:
    print(qc.draw("text"))

# ─────────────────────────────────────────────
# Step 2 – Simulate Teleportation
# ─────────────────────────────────────────────
sim = AerSimulator()
tqc = transpile(qc, sim)

# Run simulation
result = sim.run(tqc, shots=1024).result()
counts = result.get_counts()

plot_histogram(counts)
plt.title("Measurement Outcomes (Alice → Bob Classical Bits)")
plt.show()

# ─────────────────────────────────────────────
# Step 3 – Verify Bob’s Qubit State
# ─────────────────────────────────────────────
# Create pure quantum circuit (no measurements)
qc_pure = QuantumCircuit(3)
qc_pure.h(0)
qc_pure.t(0)
qc_pure.h(1)
qc_pure.cx(1, 2)
qc_pure.cx(0, 1)
qc_pure.h(0)
qc_pure.cx(1, 2)
qc_pure.cz(0, 2)

# Compute final statevector
final_state = Statevector.from_instruction(qc_pure)

# Extract Bob’s qubit (index 2)
bob_density = partial_trace(final_state, [0, 1])
bob_state = DensityMatrix(bob_density)

# Plot Bloch sphere
plot_bloch_multivector(bob_state)
plt.title("Bob’s Qubit After Teleportation (Teleported State)")
plt.show()

# Optional: print Bob’s teleported state
print("Bob’s teleported density matrix:\n", bob_state.data)


     ┌───┐┌───┐ ░            ░      ┌───┐ ░ ┌─┐    ░         
q_0: ┤ H ├┤ T ├─░────────────░───■──┤ H ├─░─┤M├────░───────■─
     └───┘└───┘ ░ ┌───┐      ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐ ░       │ 
q_1: ───────────░─┤ H ├──■───░─┤ X ├──────░──╫─┤M├─░───■───┼─
                ░ └───┘┌─┴─┐ ░ └───┘      ░  ║ └╥┘ ░ ┌─┴─┐ │ 
q_2: ───────────░──────┤ X ├─░────────────░──╫──╫──░─┤ X ├─■─
                ░      └───┘ ░            ░  ║  ║  ░ └───┘   
c: 2/════════════════════════════════════════╩══╩════════════
                                             0  1            
Bob’s teleported density matrix:
 [[0.5       +0.j         0.35355339-0.35355339j]
 [0.35355339+0.35355339j 0.5       +0.j        ]]


In [ ]:
# Quantum Teleportation – Final Colab-Friendly Version
!pip install qiskit qiskit-aer pylatexenc --quiet

import matplotlib
matplotlib.use("Agg")          # safer static backend
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_vector
from qiskit.quantum_info import Statevector, partial_trace, DensityMatrix
import numpy as np

# ─── Step 1 – Build teleportation circuit ─────────────────────
qc = QuantumCircuit(3, 2)
qc.h(0); qc.t(0)               # |ψ⟩ to teleport
qc.barrier()
qc.h(1); qc.cx(1, 2)           # entangle Alice-Bob
qc.barrier()
qc.cx(0, 1); qc.h(0)
qc.barrier()
qc.measure([0,1],[0,1])
qc.barrier()
qc.cx(1,2); qc.cz(0,2)
display(qc.draw("text"))       # text mode always works

# ─── Step 2 – Simulate ─────────────────────────────────────────
sim = AerSimulator()
tqc = transpile(qc, sim)
result = sim.run(tqc, shots=1024).result()
counts = result.get_counts()
print("\nMeasurement counts:", counts)
plt.figure()
plot_histogram(counts)
plt.title("Alice’s Measurement Outcomes")
plt.show()

# ─── Step 3 – Verify Bob’s qubit ───────────────────────────────
qc_pure = QuantumCircuit(3)
qc_pure.h(0); qc_pure.t(0)
qc_pure.h(1); qc_pure.cx(1,2)
qc_pure.cx(0,1); qc_pure.h(0)
qc_pure.cx(1,2); qc_pure.cz(0,2)

final_state = Statevector.from_instruction(qc_pure)
bob_density = partial_trace(final_state, [0,1])
bob_state = DensityMatrix(bob_density)

# Convert Bob’s state to Bloch-vector coordinates
bloch = [2*np.real(bob_state.data[0,1]),
         2*np.imag(bob_state.data[1,0]),
         np.real(bob_state.data[0,0]-bob_state.data[1,1])]
print("\nBloch-vector components (x,y,z):", np.round(bloch,3))

plt.figure()
plot_bloch_vector(bloch)
plt.title("Bob’s Qubit After Teleportation")
plt.show()


┌───┐┌───┐ ░            ░      ┌───┐ ░ ┌─┐    ░         
q_0: ┤ H ├┤ T ├─░────────────░───■──┤ H ├─░─┤M├────░───────■─
     └───┘└───┘ ░ ┌───┐      ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐ ░       │ 
q_1: ───────────░─┤ H ├──■───░─┤ X ├──────░──╫─┤M├─░───■───┼─
                ░ └───┘┌─┴─┐ ░ └───┘      ░  ║ └╥┘ ░ ┌─┴─┐ │ 
q_2: ───────────░──────┤ X ├─░────────────░──╫──╫──░─┤ X ├─■─
                ░      └───┘ ░            ░  ║  ║  ░ └───┘   
c: 2/════════════════════════════════════════╩══╩════════════
                                             0  1


Measurement counts: {'01': 247, '00': 249, '10': 277, '11': 251}

Bloch-vector components (x,y,z): [0.707 0.707 0.   ]
